In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
#This code is inspired in https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
def parse_url(url, date):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    #tr_elements = doc.xpath('//tr')
    tr_elements = doc.xpath('/html/body/div[1]/div[4]/div[1]/div/div[3]/table/tbody/tr')
        
    
    #Create empty list
    col=[]
    i=0#For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content().strip()
        #print ("column:", i, name)
        col.append((name,[]))
        
    #Append the date column   
    col.append(("Fecha", []))
        
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #print ("row:", j)
        
        #T is our j'th row
        T=tr_elements[j]
        #print ("row:", T)

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=2:
            break

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            #print("i:", i, "j:", j, "cell:", t)
            data=t.text_content().strip() 
            #print("i:", i, "j:", j, "data:", data)
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)                    
                except:
                    pass
            #Append the data to the empty list of the i'th column
            #print (data)
            col[i][1].append(data)
            #Increment i for the next column
            i+=1    
            
        #Append the date column   
        col[i][1].append(date)
            
    #print("col:", col)        
    
    Dict={title:column for (title,column) in col}    
    df=pd.DataFrame(Dict)
    #df.head()
    
    return df


In [3]:
# #Scrap the HTML as data frames
# frames = [parse_url("http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.9/id.263522", "2020-07-21"), 
#           parse_url("http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.9/id.263587", "2020-07-22")]

# #Concatenate the data frames
# result = pd.concat(frames)

# result.head()

# #Pivot the table
# table = pd.pivot_table(result, values='Casos', index=['Zona Básica'],
#                     columns=['Fecha'], fill_value=0)
# table.head()

In [4]:
# #Pivot the table
# table = pd.pivot_table(result, values='Casos', index=['Zona Básica'],
#                     columns=['Fecha'], fill_value=0)
# table.head()

In [5]:
# print(table)

In [29]:
#!pip install PyPDF2
def parse_pdf_url(url, date):
    import PyPDF2

    response = requests.get(url)
    my_raw_data = response.content

    with open("my_pdf.pdf", 'wb') as my_data:
        my_data.write(my_raw_data)

    page_content = ""
        
    read_pdf = PyPDF2.PdfFileReader("my_pdf.pdf")
    number_of_pages = read_pdf.getNumPages()
    #page = read_pdf.getPage(0)
    #page_content = page.extractText()
    for page_number in range(number_of_pages):   # use xrange in Py2
            page = read_pdf.getPage(page_number)
            page_content = page_content + page.extractText()

    page_content = page_content.replace("1\n \n \nCasos\n \nnotificados", "Casos notificados") 
            
    page_content = page_content.replace("TORRERO\n \nLA PAZ", "TORRERO LA PAZ")
    page_content = page_content.replace("AVENIDA \nCATALUÑA", "AVENIDA CATALUÑA")
    page_content = page_content.replace("BIESCAS\n-\nVALLE DE TENA", "BIESCAS-VALLE DE TENA ")
    page_content = page_content.replace("SANTA \nEULALIA DEL CAMPO", "SANTA EULALIA DEL CAMPO")
    page_content = page_content.replace("VALDESPARTERA\n-\nMONTECANAL", "VALDESPARTERA-MONTECANAL")
    page_content = page_content.replace("SAGASTA\n-\nRUISEÑORES", "SAGASTA-RUISEÑORES")
    page_content = page_content.replace("HUESCA CAPITAL -1 \n(PERPETUO SOCORRO", "HUESCA CAPITAL Nº 1 (PERPETUO SOCORRO)")
    page_content = page_content.replace("HUESCA CAPITAL Nº 1 \n(PERPETUO SOCORRO", "HUESCA CAPITAL Nº 1 (PERPETUO SOCORRO)")
    page_content = page_content.replace("HUESCA CAPITAL Nº 2 (SANTO \nGRIAL)", "HUESCA CAPITAL Nº 2 (SANTO GRIAL)")
    page_content = page_content.replace("MADRE \nVEDRUNA\n-\nMIRAFLORES", "MADRE VEDRUNA-MIRAFLORES")
    page_content = page_content.replace("MADRE VEDRUNA\n-\nMIRAFLORES", "MADRE VEDRUNA-MIRAFLORES")                                                                                                                    
    
    page_content = page_content.replace("\n-\n \n", "-")
    page_content = page_content.replace("Nº\n \n", "-")
    
    import re
    page_content = re.sub(r'(\d+)\n \n\d+', r'\1', page_content)
    

    print (page_content)
        
    
    page_content_split = page_content.split('\n \n')
    type(page_content_split)
        
    
    #page_content_split_filtered = list(filter(lambda x: "Zona Básica" not in x and x != "Casos" and x != " " and x != "" and x != " \n" and not "Casos" in x and not "/2020" in x, page_content_split))
    page_content_split_filtered = list(filter(lambda x: "nosticados" not in x and "notificados" not in x and "Zona Básica" not in x and x != "Casos" and x != " " and x != "" and x != " \n" and not "/2020" in x, page_content_split))
    print(page_content_split_filtered)

    elements = int((len(page_content_split_filtered))/2)
    #print(elements)
    type(elements)

    import  numpy as np
    reshaped_array = np.reshape(np.array(page_content_split_filtered), (elements, 2))

    a_dataframe = pd.DataFrame(reshaped_array, columns=["Zona Básica", "Casos"])
    a_dataframe = a_dataframe.astype({'Casos': 'int32'})

    a_dataframe = a_dataframe.assign(Fecha=[date]*elements)
    
    return a_dataframe

In [31]:
#Scrap the HTML as data frames
frames = [parse_url("http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.9/id.263522", "2020-07-21"), 
          parse_url("http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.9/id.263587", "2020-07-22"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_zonas_de_salud_240720_b207d97a%232E%23pdf", "2020-07-23"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_diagnosticados_el_24_21d693c5%232E%23pdf", "2020-07-24"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_diagnosticados_a_25_f6c24e2b%232E%23pdf", "2020-07-25"),              
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_27_f174cec5%232E%23pdf", "2020-07-26"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_28_502913f8%232E%23pdf", "2020-07-27"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_29_8534c52d%232E%23pdf", "2020-07-28")]

#Concatenate the data frames
result = pd.concat(frames)

result.head()

#Pivot the table
table = pd.pivot_table(result, values='Casos', index=['Zona Básica'],
                    columns=['Fecha'], fill_value=0)

print(table)
    

Zona Básica
 
Casos
 
DELICIAS NORTE
 
15
 
HERNAN CORTES
 
15
 
AVENIDA CATALUÑA
 
11
 
FERNANDO EL CATOLICO
 
11
 
SAGASTA-RUISEÑORES
 
9
 
SAN JOSE SUR
 
9
 
ALCAÑIZ
 
8
 
CASPE
 
8
 
DELICIAS SUR
 
8
 
FRAGA
 
8
 
CASETAS
 
7
 
MONZON URBANA
 
7
 
SAN PABLO
 
7
 
TERUEL CENTRO
 
7
 
ZARAGOZA
 
6
 
ACTUR OESTE
 
6
 
INDEPENDENCIA
 
6
 
MARIA DE HUERVA
 
6
 
TORRE RAMONA
 
6
 
UNIVERSITAS
 
6
 
ALBALATE DE CINCA
 
5
 
EJEA DE LOS CABALLEROS
 
5
 
HUESCA CAPITAL Nº 1 (PERPETUO SOCORRO))
 
5
 
LAS FUENTES NORTE
 
5
 
ROMAREDA -SEMINARIO
 
5
 
SAN JOSE CENTRO
 
5
 
TORRERO LA PAZ
 
5
 
ALIAGA
 
4
 
BINEFAR
 
4
 
CASABLANCA
 
4
 
MADRE VEDRUNA-MIRAFLORES
 
4
 
PARQUE GOYA
 
4
 
VENECIA
 
4
 
ACTUR NORTE
 
3
 
ACTUR SUR
 
3
 
ALAGON
 
3
 
ALMUDEVAR
 
3
 
HUESCA CAPITAL Nº 2 (SANTO GRIAL)
 
3
 
OLIVER
 
3
 
SANTA EULALIA DEL CAMPO
 
3
 
TAUSTE
 
3
 
TERUEL ENSANCHE
 
3
 
ZUERA
 
3
 
ARRABAL
 
2
 
BAGUENA
 
2
 
BOMBARDA
 
2
 
GRAÑEN
 
2
 
HIJAR
 
2
 
JACA
 
2
 
MEQUINENZA
 
2
 
MIRALBUENO
-

In [24]:
# #!pip uninstall tabula -y
# #!pip install tabula-py

# from tabula import read_pdf

# url_pdf = "http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_29_8534c52d%232E%23pdf"
# table_pdf = read_pdf(url_pdf)
# #table_pdf = read_pdf(url_pdf, multiple_tables=True, pages = "all")
# table_pdf

'pages' argument isn't specified.Will extract only from page 1 by default.
Error from tabula-java:
Error: Error: End-of-File, expected line




CalledProcessError: Command '['java', '-Dfile.encoding=UTF8', '-jar', '/opt/conda/envs/Python36/lib/python3.6/site-packages/tabula/tabula-1.0.3-jar-with-dependencies.jar', '--guess', '--format', 'JSON', '/home/dsxuser/.tmp/82b79a7c-eb2d-4fa6-9a25-c1d5b5d45adf.pdf']' returned non-zero exit status 1.

In [32]:
#Delete the TOTAL row, because we do not have total every day
table = table[table.index != "TOTAL"]
#Create the TOTAL row again
table = table.append(table.sum().rename('TOTAL'))

#Create the TOTAL column
total_column = table["2020-07-21"] + table["2020-07-22"] + table["2020-07-23"] + table["2020-07-24"] + table["2020-07-25"] + table["2020-07-26"] + table["2020-07-27"] + table["2020-07-28"]
table["TOTAL"] = total_column

table.sort_values(by=['TOTAL'], ascending=False).head(100)


ImportError: cannot import name 'roperator'

In [ ]:
# type(page_content)
# page_content_split = page_content.split('\n \n')
# type(page_content_split)
# page_content_split_filtered = list(filter(lambda x: x != "Zona Básica" and x != "Casos" and x != " " and x != "" and x != " \n", page_content_split))
# print(page_content_split_filtered)

# elements = int((len(page_content_split_filtered))/2)
# print(elements)
# type(elements)

# import  numpy as np
# reshaped_array = np.reshape(np.array(page_content_split_filtered), (elements, 2))

# a_dataframe = pd.DataFrame(reshaped_array, columns=["Zona Básica", "Casos"])
 
# print (a_dataframe)

In [30]:
# table[table["Zona Básica"].str.contains("SAGASTA")].head()
#list(table.columns) 

#table[table.index.str.contains("MADRE")].head()

#table[table.index != "TOTAL"].head()

# table = table.append(table.sum(numeric_only=True), ignore_index=True)



# table


parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_29_8534c52d%232E%23pdf", "2020-07-28")

Casos notificados 
el
 
2
9
/7/2020
 
 
 
Zona Básica
 
Casos
 
BAGUENA
 
49
 
MIRALBUENO
-
GARRAPINILLOS
 
29
 
FRAGA
 
20
 
UNIVERSITAS
 
19
 
DELICIAS SUR
 
18
 
SAN JOSE SUR
 
18
 
TERUEL CENTRO
 
17
 
ALBALATE DE CINCA
 
14
 
HERNAN CORTES
 
12
 
LAS FUENTES NORTE
 
12
 
ARRABAL
 
11
 
BOMBARDA
 
11
 
OLIVER
 
10
 
REBOLERIA
 
10
 
CASABLANCA
 
9
 
SAGASTA-RUISEÑORES
 
9
 
HIJAR
 
8
 
SAN JOSE CENTRO
 
8
 
VENECIA
 
8
 
AVENIDA CATALUÑA
 
7
 
DELICIAS NORTE
 
7
 
ALMOZARA
 
6
 
ROMAREDA -SEMINARIO
 
6
 
SAN PABLO
 
6
 
TORRE RAMONA
 
6
 
ACTUR NORTE
 
5
 
CASPE
 
5
 
FERNANDO EL CATOLICO
 
5
 
MARIA DE HUERVA
 
5
 
SAN JOSE NORTE
 
5
 
ZUERA
 
5
 
CALAMOCHA
 
4
 
HUESCA CAPITAL Nº 2 (SANTO GRIAL)
 
4
 
TORRERO LA 
PAZ
 
4
 
UTEBO
 
4
 
VALDESPARTERA-MONTECANAL
 
4
 
ACTUR SUR
 
3
 
ALBARRACIN
 
3
 
HUESCA CAPITAL Nº 1 (PERPETUO SOCORRO))
 
3
 
 
INDEPENDENCIA
 
3
 
MADRE VEDRUNA-MIRAFLORES
 
3
 
TERUEL ENSANCHE
 
3
 
UTRILLAS
 
3
 
ZALFONADA
 
3
 
ACTUR OESTE
 
2
 
SARIÑENA
 
2
 


ImportError: cannot import name 'is_url'

                                Zona Básica  Casos       Fecha
0                                   BAGUENA     49  2020-07-28
1              MIRALBUENO\n-\nGARRAPINILLOS     29  2020-07-28
2                                     FRAGA     20  2020-07-28
3                               UNIVERSITAS     19  2020-07-28
4                              DELICIAS SUR     18  2020-07-28
5                              SAN JOSE SUR     18  2020-07-28
6                             TERUEL CENTRO     17  2020-07-28
7                         ALBALATE DE CINCA     14  2020-07-28
8                             HERNAN CORTES     12  2020-07-28
9                         LAS FUENTES NORTE     12  2020-07-28
10                                  ARRABAL     11  2020-07-28
11                                 BOMBARDA     11  2020-07-28
12                                   OLIVER     10  2020-07-28
13                                REBOLERIA     10  2020-07-28
14                               CASABLANCA      9  202

In [22]:
!pip install camelot-py[cv]
#!pip install ghostscript

import distutils.spawn

libgs = distutils.spawn.find_executable("gs")

import camelot
#Camelot does have some additional dependencies, including GhostScript, which are listed here. Once installed, we can use Camelot similarly to tabula-py to scrape PDF tables.
file = "http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_29_8534c52d%232E%23pdf"
 
tables = camelot.read_pdf(file, pages = "1-end")

#This returns a TableList object. To access any of the tables found by index, you can do this:

# get the 0th-indexed-table table
tables[0].df

RuntimeError: Please make sure that Ghostscript is installed

In [23]:
!conda update conda


PackageNotInstalledError: Package is not installed in prefix.
  prefix: /opt/conda/envs/Python36
  package name: conda


